# Introduction

個別使用PTT creditcard版 2021年和2020年的語料做LDA，期望能夠透過非監督式的方法找出文章、句子的主題，語料處理如下：
1. 語料前處理
    - 刪除PTT標題分類（例如：[閒聊]、[發問]）
    - 刪除表情符號（字典 + unicode範圍）
    - 把多於空白刪除
    - 統一數字表示法（例如：1,000,000 -> 1000000）
    - 使用regex找出網址並刪除
    - 使用pysbd.Segmenter斷句
    - 使用CkipTagger斷詞
    - 切開過長句子
    - 合併過短句子
    - 把常見標點符號統一為全型
    - 刪除句子開頭或尾端的異常標點符號
    - 刪除 x%, x.x% 等回饋%數詞
    - 刪除純數字的詞
    
    
2. 語料過濾（句子）
    - 刪除OOV、特殊符號太多的句子
    - 刪除中文字太少的句子
    - 把標題包含「核卡」、「調額」的文章刪除
    

3. 語料過濾（單詞）
    - 刪除stopwords，參考：https://github.com/goto456/stopwords
    - 使用CkipTagger詞性標注後，刪除不是以下詞性的單詞，參考：https://github.com/ckiplab/ckiptagger/wiki/POS-Tags
        - Na: 普通名詞
        - Nb: 專有名詞
        - Nc: 地方詞
        - FW: 外文
        - VC: 動作及物動詞
    - 刪除只有一個字的單詞
    - 刪除「銀行名稱」單詞（若不刪除：則分群結果會有大量銀行名稱）
    
    
4. 訓練語料準備
    - 以文章為單位，準備n-gram語料（sliding window）
        - 目前n設定為1~5，也就是1-gram ~ 5-gram的語料會合併下去訓練
            - 實驗起來，比起用單一n-gram，更能在分群時找到更罕見的字作為主題
        - n-gram裡面必須包含至少一個銀行信用卡相關單詞，否則丟棄，參考：https://docs.google.com/spreadsheets/u/1/d/1o739ezU6qrFyw-QmbXBgu_8SlFjahOp-K5J-oddAXEM/edit#gid=1335809966

# Result

## 2021

In [4]:
lda, corpus, corpus_info, corpus_ids, dictionary = load_lda('20211123_ngram/creditcard2021')
prepare(lda, corpus_ids, dictionary)

/root/.pyenv/versions/3.8.11/envs/translator/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.107278 -0.152026       1        1  18.483039
2     -0.025740  0.229130       2        1  17.177692
1      0.071308 -0.031941       3        1  16.776347
0     -0.073323 -0.024880       4        1  16.700372
5      0.234265 -0.035894       5        1  15.572676
4     -0.099232  0.015611       6        1  15.289874, topic_info=     Term          Freq         Total Category  logprob  loglift
8      消費   8144.000000   8144.000000  Default  30.0000  30.0000
1      回饋  14532.000000  14532.000000  Default  29.0000  29.0000
257    額度   4125.000000   4125.000000  Default  28.0000  28.0000
0      現金   4843.000000   4843.000000  Default  27.0000  27.0000
84     台灣   3200.000000   3200.000000  Default  26.0000  26.0000
...   ...           ...           ...      ...      ...      ...
84     台灣    435.380941   3200.346220   Topic6  -5.2103  -0.1168
139    金額    352.660536   1858.669624   Topic6  -5.4211   0.2159
1178   申辦    301.936525   1328.539039   Topic6  -5.5763   0.3964
369    加碼    296.934348   2159.626678   Topic6  -5.5930  -0.1062
320    年費    286.606435   1285.409426   Topic6  -5.6284   0.3773

[537 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
4541      2  0.986779  10/1
185       2  0.013663   12月
185       3  0.942753   12月
185       5  0.009109   12月
185       6  0.027326   12月
...     ...       ...   ...
153       3  0.046320    點數
153       4  0.537040    點數
153       5  0.255094    點數
153       6  0.034236    點數
2124      3  0.990066    點點

[1577 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1, 6, 5])

## Example of Group 5

In [7]:
show_lines(lda, corpus, corpus_info, corpus_ids, dictionary, target=5, min_prob=0.9, show_nums=3)

topic: 5/0.950
board: creditcard
id   : M.1632587110.A.FE1
title: Re: [問題] ATM繳款錢不見了
user : walking (外匯世界 巫龍王之說)
ngram: 
ATM 繳款 錢 不見了
中國信託 分行 會 定時 卸 補鈔 ， 不管 是 什麼 情況 下 只要 拆 鈔閘 一定 會 清點 鈔票 及 金額 。 ： 所以 如果 你 確定 有 存入 ATM ， 但 沒有 紀錄 ， 一定 會 有 多餘 的 款項 是 未知 的 ， 分行 會 紀錄 該 ： 情況 。
請問 這 種 狀況 可以 跟 銀行 調 監視器 嗎 ？ ： 通常 分行 監視器 一定 會 「 至少 」 保存 三 到 六 個 月 ， 所以 請 儘速 與 該 分行 聯絡 表示 你 遇到 的 ： 問題 ， 一般來說 服務 經理會 協助 調閱 監視 影像 。
-----------------------------


## Example of Group 3

In [8]:
show_lines(lda, corpus, corpus_info, corpus_ids, dictionary, target=3, min_prob=0.9, show_nums=3)

topic: 3/0.953
board: creditcard
id   : M.1615645222.A.917
title: [新聞]信用卡奇招 買房刷訂金回饋2%
user : zithromax (zithromax)
ngram: 
年輕人 買 房 大 不易 ， 加上 不少 台商 返台 投資 ， 興建 廠房 後 也 帶來 員工 遷住 潮 等 ， 兆豐 銀行
為 首購族 全方位 設想 推出 這 張 聯名 卡 ， 倘若 至 海悅 團隊 代銷 案場 購屋 ， 訂金 可 享 2% 回饋
或是 選擇 24 期 零 利率 二 擇 一 。
加上 剩餘 自付款 ， 可 讓 想要 買 房子 的 年輕人 ， 月 繳 數萬 元 就 能 成家 ， 降低 買 房 的 難度
-----------------------------
topic: 3/0.950
board: creditcard
id   : M.1633326796.A.F76
title: [情報] 臺企銀 永續生活悠遊鈦金卡(預告)
user : pl726 (PL月見草)
ngram: 
臺企銀 ， 永續 生活 悠遊 鈦金 卡 ( 預告 )
Source 資訊藏 在 這 則 「 悠遊 聯名 卡 ( 普 卡 / 鈦金 卡 ) 」 的 換發 公告 裡面 ， 20 21 / 12/8 起 ， 將 陸續 換成 「 永續 生活 悠遊 鈦金 卡 」 。
棉花田 、 馬可 先生 、 柑仔店 、 無 毒農 、
禪風 茶樓 、 上善 蔬食 、 上善 豆家 、 養心 茶樓 、
Thomas Chien 法式 餐廳 ， 等 。
-----------------------------


---

---

## 2020

In [9]:
lda, corpus, corpus_info, corpus_ids, dictionary = load_lda('20211123_ngram/creditcard2020')
prepare(lda, corpus_ids, dictionary)

/root/.pyenv/versions/3.8.11/envs/translator/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.056423  0.011345       1        1  18.196821
5     -0.082701 -0.078530       2        1  17.583849
0     -0.076226  0.205635       3        1  16.784219
2     -0.152251 -0.104840       4        1  16.306853
3      0.155158 -0.073329       5        1  15.820403
1      0.099598  0.039718       6        1  15.307856, topic_info=    Term          Freq         Total Category  logprob  loglift
38    活動  11169.000000  11169.000000  Default  30.0000  30.0000
6     消費  11867.000000  11867.000000  Default  29.0000  29.0000
255   客服   6716.000000   6716.000000  Default  28.0000  28.0000
41    現金   7602.000000   7602.000000  Default  27.0000  27.0000
374   御璽   3882.000000   3882.000000  Default  26.0000  26.0000
..   ...           ...           ...      ...      ...      ...
38    活動    523.874133  11169.684993   Topic6  -5.3887  -1.1829
43    帳單    445.227210   4350.942737   Topic6  -5.5513  -0.4028
0     年費    406.107615   2365.951931   Topic6  -5.6433   0.1145
298   電子    387.722453   1638.856675   Topic6  -5.6896   0.4353
74    上限    401.950641   3905.843423   Topic6  -5.6536  -0.3971

[555 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
12562      1  0.010586  10/13
12562      3  0.021172  10/13
12562      4  0.010586  10/13
12562      5  0.952724  10/13
2214       1  0.834930    10月
...      ...       ...    ...
104        2  0.133184     點數
104        3  0.012400     點數
104        4  0.534115     點數
104        5  0.129051     點數
104        6  0.061540     點數

[1662 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 1, 3, 4, 2])

## Example of Group 3

In [10]:
show_lines(lda, corpus, corpus_info, corpus_ids, dictionary, target=3, min_prob=0.9, show_nums=3)

topic: 3/0.951
board: creditcard
id   : M.1581425523.A.3C6
title: Re: [問題] 彰銀 my樂 回饋通路問題
user : kinkids (kinkids)
ngram: 
2 . ： ： ： 第二 張 圖 的 意思 有些 不 清楚 ： ( 適用 範圍 ： Google Pay 手機 實體 感應 交易 ， Google Pay 線上 使用 存 於 Google Pay APP 之 彰銀 卡 面 刷卡 交易 ) ， 不 包含 Google Play Store 或 其他 自動 填入 的 實體 卡 交易 。
Google Pay 手機 實體 感應 交易 ， 是 指 ， 實體 消費 手機 nfc 刷卡 可以 理解 ： Google Pay 線上 使用 存 於 Google Pay APP 之 彰銀 卡 面 刷卡 交易 )
這 項 就是說 例如 我 用 momo /PC24 結帳 選 GP 跳轉到 Gp app 選 my 樂 卡 線上 刷卡 ， 這樣 也 是 可以 判定 回饋 嗎 ？ ： 謝謝 大家
-----------------------------
topic: 3/0.952
board: creditcard
id   : M.1603409714.A.5FB
title: [情報] 普惠金融　永豐全台首推外籍移工信用卡
user : golang (Gopher)
ngram: 
普惠 金融 ， 永豐 全 台 首推 外籍 移工 信用卡
普惠 金融 ， 永豐 全 台 首推 外籍 移工 信用卡 【 台北 訊 】 為 落實 普惠 金融 ， 永豐 銀行 即 (22) 日 推出 全 台灣 首 張 外籍 移工 專屬 信
用 卡 - 「 永豐 銀行 SEA 鈦金 商務 悠遊 卡 」 [ 1 ] 。 永豐 銀行 表示 ， 全 台灣 約 70萬 名 移工 [2 ]
在 台 工作 年限 最 長 可 至 14 年 ， 台灣 已 成為 他們 的 第二 個 故鄉 ， 為 提供 移工 更 便捷 的 在地金
-----------------------------
topic: 3/0.951
board: creditcard
id   : M.1581425523.A.3C6
title: Re: [問題] 彰銀

## Example of Group 5

In [11]:
show_lines(lda, corpus, corpus_info, corpus_ids, dictionary, target=5, min_prob=0.9, show_nums=3)

topic: 5/0.953
board: creditcard
id   : M.1583536384.A.810
title: Re: [情報] 台新昇恆昌無限卡 2020權益公告
user : flyfish2006 (flyfish)
ngram: 
2 . 航空 公司 / 旅行社 消費 項目 為 航空 公司 及 國內 各 大 旅行社 之 刷卡 交易 ， 且 刷卡 交易 名稱 需 包含 「 航空 」 、 「 旅行社 」 始 享 2.2% 回饋 。
3 . 國外 消費 項目 為 消費 地 於 國外 或 以 外幣 消費 之 刷卡 交易 ， 國內 消費 項目 為 以 新臺幣 計價 之 臺灣 地區 刷卡 交易 。
4 . 當期 新增 消費 乃 依據 國內外 消費 幣別 為 回饋 基準 ， 且 依 各 消費 類別 及 回饋 率 計算 回饋 。
-----------------------------
topic: 5/0.965
board: creditcard
id   : M.1590372577.A.058
title: [討論] 點數衍生產品知覺利益與顧客滿意度之探討
user : martin100 (哈哈哈哈哈)
ngram: 
點數 衍生 產品 知覺 利益 與 顧客 滿意度 之 探討
問卷 名稱 ： 點數 衍生 產品 知覺 利益 與 顧客 滿意度 之 探討 。 ( 填答 時間 約 五 分鐘 )
研究 目的 ： 探討 點數 聯名 卡 品牌 知覺 利益 與 顧客 滿意度 之間 的 關係 。
-----------------------------
topic: 5/0.953
board: creditcard
id   : M.1581617420.A.DF8
title: [問題] 亞太數位門市預繳款 一般消費? 最高回饋?
user : ewehs (ewehs)
ngram: 
亞太 數位 門市 預繳款 ， 一般 消費 ？ 最 高 回饋 ？
我 爬 過 文 ， 去年 8月 ， 有 人 回報 Ubear 預繳 台星 網路 門市 有 5% 本 版 #1THcowt H 不過 沒 看到 亞太 數位 門市 預繳款 的 付款 回饋 回報 ， 所以 發文 詢問 。
請教 各 位板 有 幾 個 問題
1 . 有 沒 有 版友 在 亞太 數位 門市 付費 繳 

---

---

# Code

In [1]:
import pickle
import pyLDAvis
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath
from pyLDAvis.gensim_models import prepare

pyLDAvis.enable_notebook()
EXP_ROOT = '/data/home/fintech-topic-detection/experiments'

/root/.pyenv/versions/3.8.11/envs/translator/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
import matplotlib.pyplot as plt
def plot_coherence(num_topics_range, coherence_values):
    plt.plot(num_topics_range, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [3]:
def load_lda(experiment):
    folder = f'{EXP_ROOT}/{experiment}'
    folder = folder[:-1] if folder.endswith('/') else folder
    lda = LdaModel.load(datapath(f'{folder}/model'))
    corpus = pickle.load(open(f'{folder}/corpus.p', 'rb'))
    corpus_info = pickle.load(open(f'{folder}/corpus_info.p', 'rb'))
    corpus_ids = pickle.load(open(f'{folder}/corpus_ids.p', 'rb'))
    dictionary = Dictionary.load(f'{folder}/dictionary')
    return lda, corpus, corpus_info, corpus_ids, dictionary


def show_lines(lda, corpus, corpus_info, corpus_ids, dictionary, target, min_prob, show_nums):
    count = 0
    assert len(corpus) == len(corpus_ids) == len(corpus_info)
    length = len(corpus)
    for i in range(length):
        topics = lda.get_document_topics(corpus_ids[i])
        topic, prob = sorted(topics, key=lambda x: x[1])[0]

        if topic + 1 == target and prob > min_prob:
            text = '\n'.join(corpus_info[i][4])
            print(
            f'topic: {target}/{prob:.3f}\n'
            f'board: {corpus_info[i][0]}\n'
            f'id   : {corpus_info[i][1]}\n'
            f'title: {corpus_info[i][2]}\n'
            f'user : {corpus_info[i][3]}\n'
            f'ngram: \n{text}\n'
            '-----------------------------')
            count += 1
            if count >= show_nums:
                break